In [ ]:
!git clone https://github.com/harry0703/MoneyPrinterTurbo.git
%cd MoneyPrinterTurbo
!git branch -a > branches.txt
!git log --oneline --graph --decorate --all -n 50 > commit_graph.txt
!git log --merges --oneline -n 30 > merges.txt
!git tag -n > tags.txt
!git log --pretty=format:"%h | %an | %ad | %s" --date=short -n 50 > commits_dates.txt
!git log --tags --simplify-by-decoration --pretty="format:%d %ad" --date=short > tag_dates.txt

fatal: destination path 'MoneyPrinterTurbo' already exists and is not an empty directory.
/content/MoneyPrinterTurbo


In [ ]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Limpeza de memória
gc.collect()
torch.cuda.empty_cache()

model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

# Configuração de Quantização
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("Iniciando carregamento otimizado...")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    offload_folder="offload",
    torch_dtype=torch.float16
)

# 2. Função de Inferência
def ask_qwen(prompt):
    messages = [
        {"role": "system", "content": "You are a senior DevOps engineer. Output only JSON."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        temperature=0.1,
        do_sample=False
    )

    response = tokenizer.batch_decode(generated_ids[:, model_inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]
    return response

# 3. Leitura Limitada (Para não estourar o contexto da LLM)
def read_file(filename, lines=100):
    with open(filename) as f:
        content = f.readlines()
    return "".join(content[:lines])

# Carregando dados com limites de segurança
branches = read_file("branches.txt", 20)
graph = read_file("commit_graph.txt", 100)
merges = read_file("merges.txt", 50)
tags = read_file("tags.txt", 30)
dates = read_file("commits_dates.txt", 100)
tag_dates = read_file("tag_dates.txt", 30)

# 4. Execução das Análises
prompt_branching = f"""Analyze the Git data and identify the branching model. Return ONLY JSON.
Branches: {branches}
Graph: {graph}
Merges: {merges}
"""

prompt_release = f"""Analyze the release strategy and frequency. Return ONLY JSON.
Tags: {tags}
Tag Dates: {tag_dates}
Commits: {dates}
"""

print("\n--- Analisando Branching Model ---")
print(ask_qwen(prompt_branching))

print("\n--- Analisando Release Strategy ---")
print(ask_qwen(prompt_release))

Iniciando carregamento otimizado...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Analisando Branching Model ---
```json
{
  "branching_model": "Git Flow"
}
```

--- Analisando Release Strategy ---
```json
{
  "release_strategy": "Feature-based",
  "frequency": "Weekly"
}
```
